# Step 02: Load Configuration File

This notebook loads and validates the Excel configuration file for the analysis cycle.

**Tasks:**
- Select and validate Excel configuration file
- Validate file format and required sheets
- Validate data completeness and versions
- Validate reference data existence
- Load configuration to database

In [ ]:
# Import required modules
import sys
import os
from pathlib import Path

# Determine the notebook's actual directory
if 'NOTEBOOK_DIR' in os.environ:
    notebook_dir = Path(os.environ['NOTEBOOK_DIR'])
else:
    home = Path.home()
    workspace = home / 'workspace'
    
    if workspace.exists():
        workflows = workspace / 'workflows'
        active_dirs = list(workflows.glob('Active_*'))
        
        if active_dirs:
            notebook_dir = active_dirs[0] / 'notebooks' / 'Stage_01_Setup'
        else:
            notebook_dir = Path.cwd()
    else:
        notebook_dir = Path.cwd()

print(f"Notebook directory: {notebook_dir}")

# Add workspace to path
workspace_path = notebook_dir.parent.parent.parent.parent
if str(workspace_path) not in sys.path:
    sys.path.insert(0, str(workspace_path))

from helpers import ux
from helpers.context import WorkContext
from helpers.configuration import (
    validate_configuration_file,
    load_configuration_file,
    read_configuration
)
from helpers.database import execute_query
from helpers.step import Step

## Initialize Context and Check Existing Configuration

In [ ]:
# Initialize work context
ux.header("Configuration Loading")

# Initialize context with explicit notebook directory
context = WorkContext(notebook_path=str(notebook_dir))

ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")

# Check for existing configurations
existing_configs = execute_query(
    "SELECT id, status, created_ts FROM irp_configuration WHERE cycle_id = (SELECT id FROM irp_cycle WHERE cycle_name = %s)",
    (context.cycle_name,)
)

if not existing_configs.empty:
    ux.warning("⚠ This cycle already has a configuration loaded")
    for idx, config in existing_configs.iterrows():
        ux.info(f"  - Config ID: {config['id']}, Status: {config['status']}, Created: {config['created_at']}")
    ux.warning("Loading a new configuration will replace the existing one")
    print()

## 1) Validate File Format

In [ ]:
# Get Excel configuration file path
ux.subheader("Select Configuration File")

# Suggest default path
excel_config_dir = context.cycle_directory / "files" / "excel_configuration"
ux.info(f"Expected location: {excel_config_dir}")

# Get file path from user
config_file_path = ux.text_input(
    "Enter the path to the Excel configuration file:",
    default=str(excel_config_dir / "configuration.xlsx")
)

# Convert to Path object and validate
config_path = Path(config_file_path)

if not config_path.exists():
    ux.error(f"✗ File not found: {config_path}")
    raise FileNotFoundError(f"Configuration file not found: {config_path}")

if not config_path.suffix.lower() in ['.xlsx', '.xls']:
    ux.error(f"✗ Invalid file format: {config_path.suffix}")
    raise ValueError("Configuration file must be an Excel file (.xlsx or .xls)")

# Display file information
file_size = config_path.stat().st_size / 1024  # KB
file_modified = config_path.stat().st_mtime

from datetime import datetime
file_info = [
    ["File Name", config_path.name],
    ["File Path", str(config_path)],
    ["File Size", f"{file_size:.2f} KB"],
    ["Last Modified", datetime.fromtimestamp(file_modified).strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(file_info, headers=["Property", "Value"])
ux.success("✓ Configuration file found")

## 2) Validate Required Data Present

In [ ]:
# Validate configuration file structure and content
ux.header("Configuration Validation")
ux.info("Validating configuration file structure and content...")

try:
    # Run validation (returns validation results)
    validation_results = validate_configuration_file(str(config_path))
    
    # Check if validation data is present
    if '_validation' not in validation_results:
        ux.error("✗ Validation results not found in configuration data")
        raise ValueError("Configuration validation failed to produce results")
    
    validation_data = validation_results['_validation']
    
    # Display validation summary
    ux.subheader("Validation Summary")
    
    all_success = True
    summary_rows = []
    
    for sheet_name, result in validation_data.items():
        status = result.get('status', 'UNKNOWN')
        status_icon = "✓" if status == 'SUCCESS' else "✗"
        summary_rows.append([sheet_name, status, status_icon])
        if status != 'SUCCESS':
            all_success = False
    
    ux.table(summary_rows, headers=["Sheet", "Status", "Result"])
    
    if all_success:
        ux.success("\n✓ All sheets validated successfully")
    else:
        ux.error("\n✗ Some sheets failed validation")
    
except Exception as e:
    ux.error(f"✗ Validation failed: {str(e)}")
    raise

## 3) Validate Versions (EDM, Model Profile, GeoHaz)

In [ ]:
# Display detailed validation results for sheets with errors or warnings
ux.subheader("Detailed Validation Results")

has_errors = False
has_warnings = False

for sheet_name, result in validation_data.items():
    errors = result.get('errors', [])
    warnings = result.get('warnings', [])
    
    if errors or warnings:
        ux.info(f"\n{sheet_name}:")
        
        if errors:
            has_errors = True
            ux.error(f"  Errors ({len(errors)}):")
            for error in errors:
                ux.error(f"    - {error}")
        
        if warnings:
            has_warnings = True
            ux.warning(f"  Warnings ({len(warnings)}):")
            for warning in warnings:
                ux.warning(f"    - {warning}")

if not has_errors and not has_warnings:
    ux.success("✓ No errors or warnings found")
elif has_errors:
    ux.error("\n✗ Configuration file has validation errors")
    ux.info("Please fix the errors and try again")
    raise ValueError("Configuration validation failed with errors")
elif has_warnings:
    ux.warning("\n⚠ Configuration file has warnings")
    ux.info("You may proceed, but review the warnings carefully")

## 4) Validate Existence of Reference Data

In [ ]:
# Display key configuration data for review
ux.header("Configuration Contents Preview")

# Display Metadata
if 'Metadata' in validation_results:
    ux.subheader("Metadata")
    metadata = validation_results['Metadata']
    metadata_rows = [[k, v] for k, v in metadata.items() if not k.startswith('_')]
    ux.table(metadata_rows, headers=["Key", "Value"])

# Display Databases count
if 'Databases' in validation_results:
    databases = validation_results['Databases']
    ux.info(f"\n✓ Databases: {len(databases)} configured")

# Display Portfolios count
if 'Portfolios' in validation_results:
    portfolios = validation_results['Portfolios']
    ux.info(f"✓ Portfolios: {len(portfolios)} configured")
    
    # Show portfolio names
    portfolio_names = [p.get('Portfolio', 'Unknown') for p in portfolios]
    ux.info(f"  Portfolios: {', '.join(portfolio_names)}")

# Display Analysis count
if 'Analysis Table' in validation_results:
    analyses = validation_results['Analysis Table']
    ux.info(f"✓ Analyses: {len(analyses)} configured")

# Display Moody's Reference Data
if "Moody's Reference Data" in validation_results:
    ux.subheader("\nMoody's Reference Data")
    ref_data = validation_results["Moody's Reference Data"]
    
    if 'Model Profiles' in ref_data:
        ux.info(f"✓ Model Profiles: {len(ref_data['Model Profiles'])} available")
    if 'Output Profiles' in ref_data:
        ux.info(f"✓ Output Profiles: {len(ref_data['Output Profiles'])} available")
    if 'Event Rate Schemes' in ref_data:
        ux.info(f"✓ Event Rate Schemes: {len(ref_data['Event Rate Schemes'])} available")

## Load Configuration to Database

In [ ]:
# Confirm before loading to database
ux.header("Load Configuration to Database")

if has_warnings:
    ux.warning("⚠ Configuration has warnings - proceed with caution")

# Ask for confirmation
proceed = ux.yes_no("Load this configuration to the database?", default="yes")

if not proceed:
    ux.info("Configuration loading cancelled by user")
    raise SystemExit("User cancelled configuration loading")

# Load configuration to database
ux.info("\nLoading configuration to database...")

try:
    # Get cycle ID
    cycle_result = execute_query(
        "SELECT id FROM irp_cycle WHERE cycle_name = %s",
        (context.cycle_name,)
    )
    
    if cycle_result.empty:
        raise ValueError(f"Cycle not found: {context.cycle_name}")
    
    cycle_id = cycle_result.iloc[0]['id']
    
    # Load configuration (this validates, deletes old configs, and loads new one)
    config_id = load_configuration_file(cycle_id, str(config_path))
    
    ux.success(f"\n✓ Configuration loaded successfully")
    ux.info(f"Configuration ID: {config_id}")
    
    # Retrieve and display loaded configuration info
    loaded_config = read_configuration(config_id)
    
    config_info = [
        ["Configuration ID", loaded_config['id']],
        ["Cycle ID", loaded_config['cycle_id']],
        ["Status", loaded_config['status']],
        ["Created", loaded_config['created_at'].strftime('%Y-%m-%d %H:%M:%S')]
    ]
    ux.table(config_info, headers=["Property", "Value"])
    
except Exception as e:
    ux.error(f"✗ Failed to load configuration: {str(e)}")
    raise

## Complete Step

In [ ]:
# Complete step execution tracking
ux.header("Step Completion")

try:
    step = Step(context)
    
    # Check if step was already executed
    if step.executed:
        ux.warning("⚠ This step has already been executed")
        ux.info(f"Status: {step.status}")
        
        # Allow override
        response = ux.yes_no("Do you want to re-run this step?", default="no")
        if response:
            step.start(force=True)
        else:
            ux.info("Step completion tracking skipped")
            raise SystemExit("Step already completed")
    else:
        step.start()
    
    # Log configuration loading
    step.log(f"Configuration file validated: {config_path.name}")
    step.log(f"Configuration loaded to database with ID: {config_id}")
    step.log(f"Validation status: {'SUCCESS' if all_success else 'SUCCESS_WITH_WARNINGS'}")
    
    # Complete the step
    step.complete({
        'configuration_id': config_id,
        'configuration_file': str(config_path),
        'validation_status': 'SUCCESS' if all_success else 'SUCCESS_WITH_WARNINGS',
        'has_warnings': has_warnings,
        'databases_count': len(validation_results.get('Databases', [])),
        'portfolios_count': len(validation_results.get('Portfolios', [])),
        'analyses_count': len(validation_results.get('Analysis Table', []))
    })
    
    ux.success("\n" + "="*60)
    ux.success("✓ CONFIGURATION LOADED SUCCESSFULLY")
    ux.success("="*60)
    ux.info("\nYou can now proceed to Stage 02: Data Extraction")
    
except SystemExit:
    # User chose not to re-run or cancelled
    pass
except Exception as e:
    ux.error(f"✗ Step completion failed: {str(e)}")
    if 'step' in locals():
        step.fail(str(e))
    raise